In [6]:
import numpy as np 
import pandas as pd 
import cv2
from PIL import Image
import os

In [7]:
traindf=pd.read_csv("../input/covid-chest-xray/metadata.csv")
traindf

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,205,11.0,M,55.0,COVID-19,Y,NaN,Y,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,"ITU admission, Endotracheal tube, nasogastric ...","Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
368,205,13.0,M,55.0,COVID-19,Y,NaN,Y,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,Lines and tubes suitably sited. Minor regress...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
369,205,20.0,M,55.0,COVID-19,Y,NaN,Y,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,increasing oxygen requirements. Extubated. Po...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
370,205,24.0,M,55.0,COVID-19,Y,NaN,N,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,Extubated since the prior radiograph. Partial...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN


In [8]:
traindf["imgpath"]="../input/covid-chest-xray/images/"+traindf.filename

In [9]:
train=traindf[["finding","imgpath"]]

In [11]:
labels = sorted(list(set(train['finding'])))
classes = len(labels)

class_to_num = dict(zip(labels, range(classes)))

In [12]:
data=[]
labels=[]

for i in range(len(train.imgpath)):
    if os.path.isfile(train.imgpath[i])==False:
        continue
    image=cv2.imread(train.imgpath[i])
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((224,224))
    data.append(np.array(size_image))
    labels.append(class_to_num[train.finding[i]])


In [13]:
X=np.array(data)
y=np.array(labels)

In [28]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from keras import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.losses import BinaryCrossentropy
from keras.optimizers import Adam, SGD

In [16]:
y = to_categorical(y)

In [17]:
y.shape

(351, 11)

In [18]:
X = X.astype('float32')/255

In [19]:
model = VGG16(weights= 'imagenet', include_top=False, input_shape= (224,224,3))

x = GlobalAveragePooling2D()(model.output)
out = Dense(11, activation= 'softmax')(x)
model = Model(inputs = model.input, outputs = out)

58892288/58889256 [==============================] - 0s 0us/step


In [33]:
sgd = SGD(lr=0.001)
model.compile(optimizer= sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.2, random_state = 707)

In [38]:
model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Epoch 1/10
9/9 [==============================] - 2s 215ms/step - loss: 0.5975 - accuracy: 0.7857 - val_loss: 0.7070 - val_accuracy: 0.7465
Epoch 2/10
9/9 [==============================] - 2s 215ms/step - loss: 0.5975 - accuracy: 0.7857 - val_loss: 0.7070 - val_accuracy: 0.7465
Epoch 3/10
9/9 [==============================] - 2s 212ms/step - loss: 0.5975 - accuracy: 0.7857 - val_loss: 0.7070 - val_accuracy: 0.7465
Epoch 4/10
9/9 [==============================] - 2s 214ms/step - loss: 0.5975 - accuracy: 0.7857 - val_loss: 0.7070 - val_accuracy: 0.7465
Epoch 5/10
9/9 [==============================] - 2s 223ms/step - loss: 0.5975 - accuracy: 0.7857 - val_loss: 0.7070 - val_accuracy: 0.7465
Epoch 6/10
9/9 [==============================] - 2s 220ms/step - loss: 0.5975 - accuracy: 0.7857 - val_loss: 0.7070 - val_accuracy: 0.7465
Epoch 7/10
9/9 [==============================] - 2s 215ms/step - loss: 0.5975 - accuracy: 0.7857 - val_loss: 0.7070 - val_accuracy: 0.7465
Epoch 8/10
9/9 [====